In [ ]:
!pip install -r requirements.txt

In [13]:
from azure.identity import AzureCliCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool
from dotenv import load_dotenv
import os

load_dotenv()

# CONNECTION_STRING = "<HostName>; <AzureSubscriptionId>; <ResourceGroup>; <HubName>"
CONNECTION_STRING=os.getenv("AZURE_CONNECTION_STRING")
SEARCH_CONNECTION_NAME = os.getenv("AI_SEARCH_CONNECTION_NAME") # Your search connection name
SEARCH_INDEX_NAME = os.getenv("AI_SEARCH_INDEX_NAME") # Your search index name

client = AIProjectClient.from_connection_string(
credential=AzureCliCredential(),
conn_str=CONNECTION_STRING,
)

conn_id = next(c.id for c in client.connections.list() if c.name == SEARCH_CONNECTION_NAME)

search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name=SEARCH_INDEX_NAME
)

agent = client.agents.create_agent(
    model="gpt-4o-mini", # Model Deployment Name
    name="my-assistant",
    instructions="You are a helpful assistant that answers questions using Azure AI Search docs",
    tools=search_tool.definitions,
    tool_resources=search_tool.resources,
)

thread = client.agents.create_thread()

client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="BQ means what in AZS?",
)

client.agents.create_and_process_run(
    thread_id=thread.id,
    assistant_id=agent.id,
)

messages = client.agents.list_messages(thread_id=thread.id)
for message in messages['data']:
    role = message['role'].upper()
    content = ""
    for content_item in message.get("content", []):
        content = content_item.get("text", {}).get("value", "")
    print(f"{role}: {content}")

ASSISTANT: In the context of Azure AI Search (AZS), "BQ" stands for Binary Quantization. It is a method used to compress high-dimensional vectors by representing each component of the vector as a single bit, either 0 or 1. This significantly reduces the memory footprint and speeds up vector comparison operations, which is vital for tasks such as search and retrieval【3:0†source】.
USER: BQ means what in AZS?
